# AWS Client
---

## Configure

```sh    
$ aws configure 
```

## EC2

### VPC/Subnets

**List all vpcs**
```sh
$ aws ec2 describe-vpcs
```

**Create vpc with cidr**
```sh
$ aws ec2 create-vpc --cidr-block 172.32.0.0/16
```

**Enable DNS/Hostname support for given vpc**
```sh
$ aws ec2 modify-vpc-attribute --vpc-id $vpcId --enable-dns-support "{\"Value\":true}"
$ aws ec2 modify-vpc-attribute --vpc-id $vpcId --enable-dns-hostnames "{\"Value\":true}"
```

**List all subnets**
```sh
$ aws ec2 describe-subnets
```

**List all subnets for specific vpc**
```sh
$ aws ec2 describe-subnets --filters Name=vpc-id,Values=$vpcId
```

**Create subnets within vpc**
```sh
$ aws ec2 create-subnet --vpc-id $vpcId --cidr-block 172.32.0.0/24
$ aws ec2 create-subnet --vpc-id $vpcId --cidr-block 172.32.1.0/24
```

**Create subnets with different availability zones**
```sh
$ aws ec2 create-subnet --vpc-id $vpcId --cidr-block 172.32.2.0/24 --availability-zone "us-west-1c"
```
- *Different availability zone is required for db instance*

**Describe subnet with id and cidr**
```sh
$ aws ec2 describe-subnets --filters "Name=vpc-id,Values=$vpcId" --query 'Subnets[*].{ID:SubnetId,CIDR:CidrBlock}'
```

**Create internet gateway**
```sh
$ aws ec2 create-internet-gateway
```

**Attach internet gateway to specific vpc**
```sh
$ aws ec2 attach-internet-gateway --vpc-id $vpcId --internet-gateway-id $gatewayId
```

**Create route table**
```sh
$ aws ec2 create-route-table --vpc-id $vpcId
```

**Describe route tables**
```sh
$ aws ec2 describe-route-tables --route-table-id $routeTableId
```

**Create route and attach it to route table and internet gateway**
```sh
$ aws ec2 create-route --route-table-id $routeTableId --destination-cidr-block 0.0.0.0/0 --gateway-id $gatewayId
```

**Attach route table with subnet**
```sh
$ aws ec2 associate-route-table  --subnet-id $subnetId --route-table-id $routeTableId
```

**Attach public ip when launching instance with the subnet**
```sh
$ aws ec2 modify-subnet-attribute --subnet-id $subnetId --map-public-ip-on-launch
```

### Create security group

**Create security group**
```sh
$ aws ec2 create-security-group --group-name web-sg --description "security group for my application"
```

**Create security group for specific vpc**
```sh
$ aws ec2 create-security-group --group-name mySSH --description "Security group for SSH access" --vpc-id $vpcId
$ aws ec2 create-security-group --group-name myDB --description "Security group for DB access" --vpc-id $vpcId
```

**List security groups**
```sh
$ aws ec2 describe-security-groups"
```

*list with specific vpc id*
```sh
$ aws ec2 describe-security-groups --filters "Name=vpc-id,Values=$vpcId"
```

### Authorize IP to group

**Allow ssh access to everyone**
```sh
$ aws ec2 authorize-security-group-ingress --group-name mySSH --protocol tcp --port 22 --cidr 0.0.0.0/0
```

**Allow web access to everyone**
```sh
$ aws ec2 authorize-security-group-ingress --group-name web-sg --protocol tcp --port 80 --cidr 0.0.0.0/0
```

**Allow db access to the group**
```sh
$ aws ec2 authorize-security-group-ingress --group-id $groupId --protocol tcp --port 3306
```

*Allow db access to specific security group*
```sh
$ aws ec2 authorize-security-group-ingress --group-id $groupId --source-group $groupId --protocol tcp --port 5432
```

### Create Key-pair

```sh
$ aws ec2 create-key-pair --key-name my-key --query 'KeyMaterial' --output text > ~/.ssh/my-key.pem
```

### List key-pairs
```sh
$ aws ec2 describe-key-pairs
```

### describe images
```sh
$ aws ec2 describe-images --image-ids ami-16efb076
```

*List/Filter images*
```sh
$ aws ec2 describe-images --owners 099720109477 --filters "Name=architecture,Values=x86_64" "Name=root-device-type,Values=ebs" "Name=name,Values=*ubuntu-xenial*" "Name=virtualization-type,Values=hvm"
```

*List/Filter images with limited fields*
```sh
$ aws ec2 describe-images --owners 099720109477 --filters "Name=architecture,Values=x86_64" "Name=root-device-type,Values=ebs" "Name=name,Values=*ubuntu-xenial*" "Name=virtualization-type,Values=hvm" --query "Images[*].{ID: ImageId, Desc:Description, CreationDate:CreationDate}" --output text

$ aws ec2 describe-images --owners 099720109477 --filters "Name=architecture,Values=x86_64" "Name=root-device-type,Values=ebs" "Name=name,Values=*ubuntu-xenial*" "Name=virtualization-type,Values=hvm" "Name=block-device-mapping.volume-type,Values=gp2" "Name=hypervisor,Values=xen" --query "Images[*].{ID: ImageId, Desc:Description, CreationDate:CreationDate}" --output text | grep -v UNSUPPORTED
```

### Run instance

```sh
$ aws ec2 run-instances --image-id ami-16efb076 --security-group-ids $groupId --count 1 --instance-type t2.micro --key-name my-key --query 'Instances[0].InstanceId'
```

**Run instance within a subnet**
```sh
$ aws ec2 run-instances --image-id ami-16efb076 --security-group-ids $groupId --count 1 --instance-type t2.micro --key-name my-key --subnet-id $subnetId
```

```sh
$ aws ec2 run-instances --image-id ami-09d2fb69 --instance-type t2.micro --key-name $keyName --security-group-ids $groupId1 $groupId2 --count 1 --subnet-id $subnetId
```

### Instance details
```sh
$ aws ec2 describe-instances --instance-ids $instanceId
```

*Filter by tag*
```sh
$ aws ec2 describe-instances --filter "Name=tag:Name,Values=myapp"
```

**Get public IP**
```sh
$ aws ec2 describe-instances --instance-ids $instanceId --query 'Reservations[0].Instances[0].PublicIpAddress'
```

**Filter by tag names**
```sh
$ aws ec2 describe-instances --filters Name=tag-value,Values=webapp
```

### Create tags
```sh
$ aws ec2 create-tags --resources $instanceId --tags Key=Name,Value=webapp Key=environment,Value=production
```

### Reboot Instance
```sh
$ aws ec2 reboot-instances --instance-ids $instanceId
```

## RDS
---

### Subnets
```sh
$ aws rds describe-db-subnet-groups
```

**Create subnet**
```sh
$aws rds create-db-subnet-group --db-subnet-group-name myapp-db-subnet --db-subnet-group-description "postgresql db subnet group" --subnet-ids $subnetId1 $subnetId2
```
- *Atleast two subnets are required with different availability zones*


### Security Groups
```sh
$ aws rds describe-db-security-groups
```

### Create db security group
```sh
$ aws rds create-db-security-group --db-security-group-name sg-app-db --db-security-group-description "security group for application db"
```

### Describe engines
```sh
$ aws rds describe-db-engine-versions --engine postgres
```

*List engine instance class*
```sh
$ aws rds describe-orderable-db-instance-options  --engine postgres --engine-version 9.6.3 --query 'OrderableDBInstanceOptions[*].DBInstanceClass' --output text 
```

### Create instance
```sh
$ aws rds create-db-instance --db-name postgresql --db-instance-identifier my-db-pgsql --allocated-storage 5 --db-instance-class db.t2.small --engine postgres --master-username $userName --master-user-password $myPassword --vpc-security-group-ids $securityGroupId --db-subnet-group-name $subnetGroupName
```

### Describe instances
```sh
$ aws rds describe-db-instances
```

### Delete instance
```sh
$ aws rds delete-db-instance --skip-final-snapshot --db-instance-identifier mysql-small-db
```